In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

os.environ["VISION_KEY"] = "EG1ieEDO7QOpKmhTVhHNZ1FaWoBDvsTqH8SsxiIewGFCttaPpmnuJQQJ99BCAC5RqLJXJ3w3AAAFACOGNK2e"
os.environ["VISION_ENDPOINT"] = "https://munteanumihai.cognitiveservices.azure.com/"

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [2]:
##img = open("test1.png", "rb")
#img = open("test2.jpeg", "rb")
img = open("Untitled.png","rb")
#img = open("ocr.png","rb")
read_response = computervision_client.read_in_stream(
    image=img,
    mode="Printed",
    raw=True
)
# print(read_response.as_dict())

operation_id = read_response.headers['Operation-Location'].split('/')[-1]
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
result = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            result.append(line.text)

print()

ComputerVisionOcrErrorException: Operation returned an invalid status code 'PermissionDenied'

In [10]:
# get/define the ground truth
#groundTruth = ["Google Cloud", "Platform"]
#groundTruth = ["Succes in rezolvarea", "tEMELOR la", "LABORAtoaree de", "Inteligenta Artificiala!"]
groundTruth=["Seat","Ateca","2019"]
#groundTruth=["Seat","Vw","Ateca","Laborator","2019"]
#groundTruth=["Seat Ateca","Vlad Mutu"]

# compute the performance
noOfCorrectLines = sum(i == j for i, j in zip(result, groundTruth))
print(noOfCorrectLines)

1


de verificat cu distantaHamming daca cuvintele din poza sunt la fel cu cele din groundTruth
Iaccard: (A intersectat cu B) / (A reunit cu B)

2. pun eu unde se afla fiecare cuvant si trebuie sa-mi dea la fel
3. cum as putea face sa identifice mai bine scrisul de mana, (de gandit si scris ca comentariu)

<h3>1. A: Hamming: pozitia caracterelor</h3>

In [11]:
from scipy.spatial.distance import hamming

total_distance = 0
for gt_word, res_word in zip(groundTruth, result):
    max_len = max(len(gt_word), len(res_word))
    gt_word = gt_word.ljust(max_len)
    res_word = res_word.ljust(max_len)
    distance = hamming(list(gt_word), list(res_word)) * max_len
    print(f"Hamming distance between '{gt_word.strip()}' and '{res_word.strip()}': {distance}")
    total_distance += distance

print(f"Total Hamming distance: {total_distance}")

Hamming distance between 'Seat' and 'Seat': 0.0
Hamming distance between 'Ateca' and 'Jeg': 5.0
Hamming distance between '2019' and 'Ateca': 5.0
Total Hamming distance: 10.0


<h3>1. B: Jaccard: absenta sau prezenta caracterelor</h3>


In [12]:
from sklearn.metrics import jaccard_score
import numpy as np

total_jaccard = 0
for gt_word, res_word in zip(groundTruth, result):
    gt_set = set(gt_word)
    res_set = set(res_word)
    union_len = len(gt_set.union(res_set))
    intersection_len = len(gt_set.intersection(res_set))
    if union_len != 0:
        jaccard_index = intersection_len / union_len
    else:
        jaccard_index = 0
    print(f"Jaccard index between '{gt_word}' and '{res_word}': {jaccard_index}")
    total_jaccard += jaccard_index

print(f"Total Jaccard index: {total_jaccard}")

Jaccard index between 'Seat' and 'Seat': 1.0
Jaccard index between 'Ateca' and 'Jeg': 0.14285714285714285
Jaccard index between '2019' and 'Ateca': 0.0
Total Jaccard index: 1.1428571428571428


<h3> 2 </h3>

In [13]:
import cv2

result2 = []
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(f"Text: {line.text}")
            print(f"Bounding Box: {line.bounding_box}")
            result2.append((line.text, line.bounding_box))

print()

# expected_locations = {
#     "Succes in rezolvarea": "top-left",
#     "tEMELOR la": "top-right",
#     "LABORAtoaree de": "bottom-left",
#     "Inteligenta Artificiala!": "bottom-right"
# }
expected_locations = {
    "Seat": "top-left",
    "Ateca": "center",
    "2019": "bottom-right"
}
# expected_locations = {
#     "Seat": "top-left",
#     "Vw": "top-center",
#     "Ateca": "center",
#     "2019": "bottom-right",
#     "Laborator": "bottom-center"
# }


def determine_location(bounding_box, image_width, image_height):
    x_coords = [bounding_box[i] for i in range(0, len(bounding_box), 2)]
    y_coords = [bounding_box[i+1] for i in range(0, len(bounding_box), 2)]
    avg_x = sum(x_coords) / len(x_coords) / image_width
    avg_y = sum(y_coords) / len(y_coords) / image_height

    if avg_x < 0.33 and avg_y < 0.33:
        return "top-left"
    elif 0.66 > avg_x >= 0.33 > avg_y:
        return "top-center"
    elif avg_x >= 0.66 and avg_y < 0.33:
        return "top-right"
    elif avg_x < 0.33 <= avg_y < 0.66:
        return "left-center"
    elif avg_x >= 0.66 > avg_y >= 0.33:
        return "right-center"
    elif avg_x < 0.33 and avg_y >= 0.66:
        return "bottom-left"
    elif 0.33 <= avg_x < 0.66 <= avg_y:
        return "bottom-center"
    elif avg_x >= 0.66 and avg_y >= 0.66:
        return "bottom-right"
    else:
        return "center"

image = cv2.imread("Untitled2.jpg")
image_height, image_width = image.shape[:2]

for text, bounding_box in result2:
    detected_location = determine_location(bounding_box, image_width, image_height)
    expected_location = expected_locations.get(text, "unknown")
    print(f"Text: '{text}'")
    print(f"Detected Location: {detected_location}")
    print(f"Expected Location: {expected_location}")
    print(f"Match: {detected_location == expected_location}")
    print()

Text: Seat
Bounding Box: [60.0, 106.0, 231.0, 97.0, 236.0, 219.0, 65.0, 234.0]
Text: Jeg
Bounding Box: [116.0, 173.0, 219.0, 196.0, 210.0, 245.0, 112.0, 225.0]
Text: Ateca
Bounding Box: [626.0, 501.0, 953.0, 557.0, 935.0, 659.0, 609.0, 606.0]
Text: 2019
Bounding Box: [1061.0, 1022.0, 1379.0, 1028.0, 1376.0, 1140.0, 1058.0, 1133.0]

Text: 'Seat'
Detected Location: top-left
Expected Location: top-left
Match: True

Text: 'Jeg'
Detected Location: top-left
Expected Location: unknown
Match: False

Text: 'Ateca'
Detected Location: center
Expected Location: center
Match: True

Text: '2019'
Detected Location: bottom-right
Expected Location: bottom-right
Match: True



<h3>3</h3>
la exemplul cu poza mea imi adauga un cuvant in plus, si ar putea ca in loc sa-l adauge, sa verifice daca restul se potrivesc si atunci sa nu-l ia in considerare. Sau in lista de groundTruth sa le verifice pe toate, nu pe rand, si sa observe ca toate se potrivesc dar a gasit un cuvant in plus
